# Creación y entrenamiento: Modelo híbrido y búsqueda multimodal

En esta fase, convertiremos los datos limpios de las transacciones, clientes y catálogos en un sistema de inteligencia artificial funcional. Nuestro objetivo es construir el motor de recomendación central que impulsará la aplicación. Para ello, utilizamos un enfoque híbrido que combina el historial de compra de los clientes (**Filtrado Colaborativo**) con las características detalladas del producto (**Recomendación Basada en Contenido**).

Adicionalmente, empleamos la Inteligencia Artificial de Oracle Cloud (**OCI Vision** y **OCI Generative AI**) para personalizar al chatbot de capacidades avanzadas:

1. **Búsqueda Multimodal**: Utilizamos OCI Vision para procesar todas las imágenes del catálogo y extraer etiquetas descriptivas, haciendo posible la búsqueda por imagen.

2. **Búsqueda Semántica** (RAG): Generamos Embeddings Semánticos (vectores numéricos de significado) con OCI Generative AI para permitir que el chatbot comprenda consultas de lenguaje natural complejo y recupere productos basándose en el significado y no solo en palabras clave exactas.

El resultado final será un modelo capaz de generar puntuaciones de interés personalizadas y de responder a consultas de texto y de imagen en tiempo real, que detallaremos para la siguiente fase de Construcción del Chatbot.

### Configuración del Entorno y Conexión a OCI

Inicializamos el entorno de desarrollo y establecemos la conectividad con los servicios de Oracle Cloud Infrastructure (OCI) que utilizaremos:
1. Autonomous Database (ADB): para los datos
2. OCI Vision / OCI Generative AI: para la inteligencia.

In [ ]:
# Instalamos las librerias necesarias
!pip install oci
!pip install oracledb
# !pip install scikit-surprise joblib
# !pip install "numpy<2" --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2555841 sha256=f13cd275c3e07ae0f3c528e390e16a098586facc75fdf5b9750fa1c06887ab36
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 115.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.

In [ ]:
#Realizamos las importaciones correspondientes
import oci
import pandas as pd
import requests
import oracledb
import re
import array
import time
import pandas as pd
# from surprise import SVD, Dataset, Reader, accuracy
# from surprise.model_selection import train_test_split
import joblib
import os
import numpy as np

In [ ]:
oracledb.defaults.fetch_lobs = False

 ### Inicialización de Clientes OCI


Definimos las credenciales necesarias y cargamos el cliente de **OCI Vision** y de **Object Storage**. Es crucial que el archivo de configuración de OCI esté disponible en este entorno para la autenticación automática.

In [ ]:
# Configuración de OCI
NAMESPACE = 'ax5grcoay8ni'
BUCKET_NAME = 'Team3'
ARTICLES_FILE = 'articles_modified.csv'

# Cargamos la configuracion y los clientes (object storage y vision)
try:
    config = oci.config.from_file("config", "DEFAULT")
    # Cliente de Object Storage (para obtener el CSV)
    object_storage_client = oci.object_storage.ObjectStorageClient(config)
    # Cliente de Vision (para procesar imágenes)
    ai_vision_client = oci.ai_vision.AIServiceVisionClient(config)
    print("Clientes de OCI Vision y Object Storage inicializados.")
except Exception as e:
    print(f"Error al inicializar clientes: {e}")

Clientes de OCI Vision y Object Storage inicializados.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Función de Consulta a ADB


Definimos la función query. Utilizaremos esta función para encapsular la conexión a Autonomous Database (ADB), garantizando que las credenciales estén correctas y que la conexión se abra y se cierre con cada consulta, facilitando la ejecución de comandos como SELECT, INSERT, etc.

In [ ]:
def query(sql, params=None):
    connection = oracledb.connect(
        user="ADMIN",
        password="TecMonterreyTeam3",
        dsn="team3vectordatabase_high",
        config_dir="/content/wallet"
    )

    with connection.cursor() as cursor:
        cursor.execute(sql, params or {})
        result = cursor.fetchall()

    connection.close()
    return result


In [ ]:
rows = query('SELECT COUNT(*) FROM "ADMIN"."ARTICLES_MODIFIED"')
for row in rows:
    print(row)

(105542,)


# Subfase: Extracción de Features de Visión (OCI Vision)

Ya que previamente cargamos nuestros datasets (ARTICLES_MODIFIED, CUSTOMERS_MODIFIED, TRANSACTIONS_MODIFIED) en ADB, ahora nos enfocamos en el enriquecimiento de contenido utilizando la inteligencia artificial de OCI Vision. El objetivo es obtener etiquetas de clasificación de las imágenes de los productos.

### 2.1. Extracción Dinámica de URLs y Creación de la Tabla Destino


Para procesar las 105,542 imágenes, primero ejecutamos la función `fetch_articles_for_vision`. Esta función realiza dos tareas críticas:


1. Consulta la URL de la imagen para cada artículo desde ADB.


2. Extrae dinámicamente el Namespace y el Bucket Name (ya que las imágenes están distribuidas en tres buckets diferentes), corrigiendo así el problema de ruteo encontrado en la fase de prueba.

In [ ]:
def fetch_articles_for_vision():
    """
    1. Consulta ADB para obtener IDs y URLs.
    2. Extrae el 'object_name', el 'bucket_name' y el 'namespace' de la URL.
    3. Devuelve un DataFrame con las ubicaciones completas.
    """
    print("Iniciando consulta a ADB para obtener IDs y URLs...")

    # Consulta a la tabla ARTICLES_MODIFIED que contiene la URL generada
    sql = 'SELECT "article_id", "img_url_Team3" FROM "ADMIN"."ARTICLES_MODIFIED"'

    # Llamamos a la función query que ya definieron
    rows = query(sql)

    # Creamos un DataFrame para fácil manipulación
    df_articles = pd.DataFrame(rows, columns=['article_id', 'img_url'])

    # Expresión regular para extraer NAMESPACE, BUCKET y OBJECT NAME
    # La URL tiene el formato: .../n/{namespace}/b/{bucket}/o/{object_name}
    regex = r'/n/([^/]+)/b/([^/]+)/o/(.*)'

    # Aplicamos la extracción con regex
    extraction_results = df_articles['img_url'].apply(
        lambda url: re.search(regex, url).groups() if re.search(regex, url) else (None, None, None)
    )

    # Asignamos las columnas extraídas
    df_articles['namespace_name'] = [res[0] for res in extraction_results]
    df_articles['bucket_name_dynamic'] = [res[1] for res in extraction_results]
    df_articles['object_name'] = [res[2] for res in extraction_results]

    print(f"Extracción dinámica completada. {len(df_articles)} artículos listos.")
    # El DataFrame ahora tiene la ubicación completa de cada objeto.
    return df_articles.dropna(subset=['object_name'])

In [ ]:
# Cargamos la lista de artículos con los campos de bucket/namespace
articles_to_process = fetch_articles_for_vision()
print(articles_to_process[['article_id', 'namespace_name', 'bucket_name_dynamic', 'object_name']].head())

### 2.2. Definición de la Lógica de Inserción y la Estructura DDL


Creamos la función `execute_non_query` para manejar comandos DML/DDL (inserción y creación de tablas). Posteriormente, la utilizamos para crear la tabla **ARTICLE_VISION_FEATURES** en ADB. Esta tabla almacenará el `article_id` y las etiquetas de visión en formato de string (vision_labels_json).

In [ ]:
def execute_non_query(sql, params=None):
    """Ejecuta un comando DDL o DML (INSERT, UPDATE) y hace commit."""
    connection = oracledb.connect(
        user="ADMIN",
        password="TecMonterreyTeam3",
        dsn="team3vectordatabase_high",
        config_dir="/content/wallet"
    )
    try:
        with connection.cursor() as cursor:
            cursor.execute(sql, params or {})
            connection.commit()
        return True
    except Exception as e:
        print(f"Error al ejecutar DML/DDL: {e}")
        return False
    finally:
        connection.close()

In [ ]:
# Creación de la tabla en la base de datos para almacenar la descripción generada por OCI Vision
# execute_non_query("CREATE TABLE ARTICLE_VISION_FEATURES (\"article_id\" NUMBER NOT NULL, \"vision_labels_json\" VARCHAR2(4000), CONSTRAINT PK_ARTICLE_VISION PRIMARY KEY (\"article_id\"))")
# print("Tabla 'ARTICLE_VISION_FEATURES' lista o previamente existente.")

### 2.3. Lógica de Envoltura para OCI Vision


Definimos la función `call_oci_vision_and_format`. Esta es nuestra interfaz con la API de OCI Vision. Es fundamental que esta función use los parámetros dinámicos de Namespace y Bucket Name obtenidos en el paso 2.1 para localizar correctamente cada imagen, sin importar en qué bucket se encuentre.

In [ ]:
def call_oci_vision_and_format(article_id, object_name, bucket_name, namespace_name):
    """Llama a OCI Vision usando los parámetros dinámicos de ubicación."""
    try:
        # Configuración de la ubicación de la imagen en Object Storage
        image_location = oci.ai_vision.models.ObjectStorageImageDetails(
            source="OBJECT_STORAGE",
            # Parametros de cada bucket
            namespace_name=namespace_name,
            bucket_name=bucket_name,
            object_name=object_name
        )

        features = [
            oci.ai_vision.models.ImageClassificationFeature(max_results=5)
        ]

        image_analysis_details = oci.ai_vision.models.AnalyzeImageDetails(
            features=features,
            image=image_location
        )

        analyze_image_response = ai_vision_client.analyze_image(
            analyze_image_details=image_analysis_details
        )

        # Extracción y Formateo de Etiquetas
        labels = [l.name for l in analyze_image_response.data.labels]
        labels_str = ", ".join(labels)

        return (article_id, labels_str)

    except Exception as e:
        print(f"Error procesando artículo {article_id} ({bucket_name}/{object_name}): {e}")
        return (article_id, "ERROR_VISION")

### 2.4. Función de Control de Procesamiento y Reanudación


La función `fetch_pending_articles` nos permite retomar el trabajo en caso de interrupciones o fallos de conexión. Consultando el progreso guardado en ARTICLE_VISION_FEATURES, podemos generar una lista de artículos pendientes. La función process_vision_batch utiliza este filtro para reanudar el trabajo de manera eficiente, realizando inserciones masivas cada 500 artículos.

In [ ]:
def process_vision_batch(df_articles, batch_size=500):
    """
    Procesa el DataFrame de artículos en lotes, llama a OCI Vision y
    almacena los resultados masivamente en ADB.
    """
    total_articles = len(df_articles)
    results_to_insert = []

    insert_sql = 'INSERT INTO ARTICLE_VISION_FEATURES ("article_id", "vision_labels_json") VALUES (:1, :2)'

    connection = oracledb.connect(
        user="ADMIN",
        password="TecMonterreyTeam3",
        dsn="team3vectordatabase_high",
        config_dir="/content/wallet"
    )

    print(f"Iniciando el procesamiento en lotes de {total_articles} artículos pendientes...")

    try:
        with connection.cursor() as cursor:
            # Iteramos usando iterrows() en el DataFrame
            for i, row in df_articles.iterrows():

                # Llamada al API de Vision con los 4 argumentos
                result = call_oci_vision_and_format(
                    row['article_id'],
                    row['object_name'],
                    row['bucket_name_dynamic'],
                    row['namespace_name']
                )

                if result and result[1] != "ERROR_VISION":
                    results_to_insert.append(result)

                # Ejecutar la inserción cada vez que se llena un lote
                if (i + 1) % batch_size == 0 or (i + 1) == total_articles:
                    if results_to_insert:
                        cursor.executemany(insert_sql, results_to_insert)
                        connection.commit()
                        print(f"   [Lote {i // batch_size + 1}] Insertados/Actualizados {len(results_to_insert)} artículos. Total procesado: {i + 1}")
                        results_to_insert = []

        print(f"Proceso de OCI Vision completado. Resultados almacenados en ARTICLE_VISION_FEATURES.")

    except Exception as e:
        print(f"CRÍTICO: Fallo en el ciclo de procesamiento: {e}")
        connection.rollback()
    finally:
        connection.close()

In [ ]:
def fetch_pending_articles(df_all_articles):
    """
    Identifica qué artículos aún NO tienen features de Vision en ADB.
    """
    print("Buscando artículos ya procesados en ADB...")

    # Consulta para obtener todos los article_id que ya están en la tabla de features
    sql_processed = 'SELECT "article_id" FROM "ADMIN"."ARTICLE_VISION_FEATURES"'

    # Utilizamos la función query
    processed_rows = query(sql_processed)

    # Convertir a un set para búsquedas rápidas
    processed_ids = {row[0] for row in processed_rows}

    print(f"Se encontraron {len(processed_ids)} artículos ya procesados.")

    # Filtramos el DataFrame original para obtener solamente los que no han sido cargados
    pending_articles = df_all_articles[~df_all_articles['article_id'].isin(processed_ids)]

    print(f"{len(pending_articles)} artículos pendientes de procesar.")

    return pending_articles

### 2.5. Ejecución y Reanudación del Procesamiento Masivo


Reanudamos la ejecución del proceso. Utilizamos `fetch_pending_articles` para filtrar la lista completa (**articles_to_process**) y obtener únicamente los artículos que aún no tienen etiquetas. Esto nos asegura que solo procesemos los ítems restantes, partiendo del último registro guardado en la base de datos.

In [ ]:
# articles_pending = fetch_pending_articles(articles_to_process)

Buscando artículos ya procesados en ADB...
✅ Se encontraron 104832 artículos ya procesados.
⏳ 710 artículos pendientes de procesar.


In [ ]:
# process_vision_batch(articles_pending, batch_size=500)

# Subfase: Generación de Embeddings

Dado que el procesamiento de Vision está completo y tenemos las etiquetas en `ARTICLE_VISION_FEATURES`, ahora prepararemos el entorno para el entrenamiento. Esto se divide en dos ramas: el **Filtrado Colaborativo** (CF) y los **Embeddings** (RAG/Contenido).

### 3.1. Preparación de Features de Interacción (Filtrado Colaborativo - CF)


Para el componente de Filtrado Colaborativo del modelo híbrido, necesitamos un dataset que resuma la interacción entre clientes y artículos. En este paso, crearemos la tabla `INTERACTIONS_FOR_CF` en ADB, la cual agrupa las transacciones y cuenta la frecuencia de compra (purchase_count), sirviendo como nuestra métrica de preferencia implícita.



```
CREATE TABLE INTERACTIONS_FOR_CF AS
SELECT
    "customer_id",
    "article_id",
    COUNT(*) AS "purchase_count"
FROM "ADMIN"."TRANSACTIONS_MODIFIED"
GROUP BY "customer_id", "article_id";

```





```
CREATE INDEX IDX_CF_KEYS ON INTERACTIONS_FOR_CF ("customer_id", "article_id");
```



### 3.2. Exportación del Dataset de Interacciones a OCI Object Storage


Una vez creada la tabla en ADB, la exportamos directamente a un archivo Parquet en OCI Object Storage. Esto se hace porque OCI Data Science (ODS) consume datos de entrenamiento de manera más eficiente desde Object Storage, y el formato Parquet es ideal para grandes volúmenes de datos tabulares.



```
BEGIN
  DBMS_CLOUD.EXPORT_DATA(
    credential_name => 'OCI_STORE_CRED',
    file_uri_list   => 'https://objectstorage.us-chicago-1.oraclecloud.com/n/ax5grcoay8ni/b/Team3/o/interactions_for_cf.parquet',
    format         => JSON_OBJECT('type' VALUE 'parquet'),
    query          => 'SELECT * FROM INTERACTIONS_FOR_CF'
  );
END;
/

```



### 3.3. Generación de Embeddings Semánticos (RAG)
El objetivo de esta fase es convertir el contenido enriquecido de los artículos (texto descriptivo + etiquetas de OCI Vision) en vectores numéricos densos (Embeddings) que capturen su significado semántico. Estos vectores son esenciales para que el chatbot pueda realizar búsquedas complejas en lenguaje natural (RAG) y comprender consultas como: "Muéstrame un vestido casual para el verano."


### 3.3.A. Unificación del Contenido y Creación de la Fuente de Datos

Ejecutamos la sentencia SQL para unificar la descripción textual original con las etiquetas de OCI Vision en una única tabla, `ARTICLE_CONTENT_RAG`. Utilizamos un JOIN para combinar ARTICLES_MODIFIED y ARTICLE_VISION_FEATURES y creamos la columna "unified_text".



```
-- Eliminar tabla si existe
BEGIN
  EXECUTE IMMEDIATE 'DROP TABLE ARTICLE_CONTENT_RAG';
EXCEPTION
  WHEN OTHERS THEN
    IF SQLCODE != -942 THEN RAISE; END IF;
END;
/

-- Crear tabla unificada
CREATE TABLE ARTICLE_CONTENT_RAG AS
SELECT
    A."article_id",
    A."description_vector_rag" ||
    ' | VISION TAGS: ' ||
    COALESCE(V."vision_labels_json", 'sin etiquetas visuales') AS "unified_text"
FROM "ADMIN"."ARTICLES_MODIFIED" A
LEFT JOIN "ADMIN"."ARTICLE_VISION_FEATURES" V
ON A."article_id" = V."article_id";

-- Crear índice sobre la clave
CREATE INDEX IDX_RAG_ID ON ARTICLE_CONTENT_RAG ("article_id");

```



### 3.3.B. Inicialización del Cliente de OCI Generative AI y Lógica de Generación

En este paso, inicializamos el cliente de OCI Generative AI y definimos la lógica para comunicarnos con la API de Embeddings, lo que nos permite procesar todos los textos en lotes y convertirlos en vectores.

Para acceder a los modelos de Embeddings (como cohere.embed-english-v3.0), debemos inicializar el cliente de OCI Generative AI. Esto implica configurar el endpoint y el Compartment ID de nuestro arrendamiento.

In [ ]:
GENAI_ENDPOINT = 'https://inference.generativeai.us-chicago-1.oci.oraclecloud.com'
GENAI_COMPARTMENT_ID = 'ocid1.compartment.oc1..aaaaaaaatjpti23mkvyvggtpv6kqujtabcigjaswynnfmhvjdjtkeaqfwakq' # Compartimento creado por el equipo

# Inicializar el cliente de OCI Generative AI
try:
    genai_client = oci.generative_ai.GenerativeAiClient(
        config=config,
        service_endpoint=GENAI_ENDPOINT
    )
    print("Cliente de OCI Generative AI inicializado.")
except Exception as e:
    print(f"Error al inicializar cliente Gen AI: {e}")

Cliente de OCI Generative AI inicializado.


Antes de comenzar, realizamos un ajuste en la configuración del driver de Oracle (python-oracledb) para asegurar que las columnas de tipo CLOB (como unified_text) se recuperen directamente como texto plano.

Esto es necesario porque, por defecto, Oracle devuelve los CLOBs como objetos LOB que requieren que la conexión permanezca abierta para poder leerse. Como nuestro flujo consulta los datos y cierra la conexión inmediatamente, configuramos el driver para traerlos ya como string.

La función fetch_and_generate_embeddings se encarga de:
1) Extraer el article_id y el campo "unified_text" desde nuestra tabla ARTICLE_CONTENT_RAG en ADB.
2) Dividir los textos en lotes y llamar a la API de OCI Generative AI para generar embeddings semánticos, aplicando truncado automático cuando el texto excede el límite del modelo.
3) Manejar fallos por lote sin detener el proceso completo (registrando None cuando ocurre un error) y controlar la carga al servicio con pausas cortas entre batches.
4) Devolver un DataFrame con el article_id y su embedding_vector asociado, listo para la inserción.

In [ ]:
genai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(
    config=config,
    service_endpoint=GENAI_ENDPOINT
)

EMBEDDING_MODEL = "cohere.embed-english-v3.0"
MAX_BATCH_SIZE = 90

def fetch_and_generate_embeddings(batch_size=MAX_BATCH_SIZE, log_every_batch=True):
    print("1. Extrayendo contenido unificado de ADB...")
    sql = 'SELECT "article_id", "unified_text" FROM "ADMIN"."ARTICLE_CONTENT_RAG"'
    rows = query(sql)

    df_content = pd.DataFrame(rows, columns=['article_id', 'unified_text'])
    texts = df_content['unified_text'].tolist()
    article_ids = df_content['article_id'].tolist()

    total = len(texts)
    print(f"Total de artículos para vectorizar: {total}")

    all_embeddings = []
    start_all = time.time()
    ok_batches = 0
    fail_batches = 0

    # batches reales
    num_batches = (total + batch_size - 1) // batch_size

    for b, i in enumerate(range(0, total, batch_size), start=1):
        batch_texts = texts[i:i + batch_size]

        start_batch = time.time()
        try:
            embed_details = oci.generative_ai_inference.models.EmbedTextDetails(
                inputs=batch_texts,
                compartment_id=GENAI_COMPARTMENT_ID,
                truncate="END",
                input_type="SEARCH_DOCUMENT",
                serving_mode=oci.generative_ai_inference.models.OnDemandServingMode(
                    model_id=EMBEDDING_MODEL
                )
            )

            response = genai_inference_client.embed_text(
                embed_text_details=embed_details
            )

            all_embeddings.extend(response.data.embeddings)
            ok_batches += 1

            if log_every_batch:
                elapsed = time.time() - start_batch
                done = min(i + batch_size, total)
                print(f"Batch {b}/{num_batches} | {done}/{total} textos | "
                      f"{elapsed:.2f}s batch")

            # micro-sleep para no saturar
            time.sleep(0.2)

        except Exception as e:
            fail_batches += 1
            all_embeddings.extend([None] * len(batch_texts))

            if log_every_batch:
                done = min(i + batch_size, total)
                print(f"⚠️ Batch {b}/{num_batches} FALLÓ | {done}/{total} textos | error: {e}")

            time.sleep(5)

    df_embeddings = pd.DataFrame({
        'article_id': article_ids,
        'embedding_vector': all_embeddings
    })

    good = df_embeddings['embedding_vector'].notna().sum()
    print("\n---- Resultados ----")
    print(f"Embeddings generados: {good}/{total}")
    print(f"Batches OK: {ok_batches} | Batches fallidos: {fail_batches}")

    return df_embeddings

df_embeddings = fetch_and_generate_embeddings()

1. Extrayendo contenido unificado de ADB...
Total de artículos para vectorizar: 105542
Batch 1/1173 | 90/105542 textos | 1.58s batch
Batch 2/1173 | 180/105542 textos | 0.74s batch
Batch 3/1173 | 270/105542 textos | 0.69s batch
Batch 4/1173 | 360/105542 textos | 0.68s batch
Batch 5/1173 | 450/105542 textos | 0.67s batch
Batch 6/1173 | 540/105542 textos | 0.68s batch
Batch 7/1173 | 630/105542 textos | 0.58s batch
Batch 8/1173 | 720/105542 textos | 0.71s batch
Batch 9/1173 | 810/105542 textos | 0.76s batch
Batch 10/1173 | 900/105542 textos | 0.60s batch
Batch 11/1173 | 990/105542 textos | 0.60s batch
Batch 12/1173 | 1080/105542 textos | 0.69s batch
Batch 13/1173 | 1170/105542 textos | 0.62s batch
Batch 14/1173 | 1260/105542 textos | 0.69s batch
Batch 15/1173 | 1350/105542 textos | 0.72s batch
Batch 16/1173 | 1440/105542 textos | 0.74s batch
Batch 17/1173 | 1530/105542 textos | 0.62s batch
Batch 18/1173 | 1620/105542 textos | 0.67s batch
Batch 19/1173 | 1710/105542 textos | 0.60s batch
Bat

### 3.3.C. Almacenamiento en Autonomous Database (Vector Store)

Aprovechamos la capacidad nativa de Base de Datos Vectorial de ADB. Creamos una tabla con el tipo de dato VECTOR y realizamos una carga masiva de los embeddings generados, lo que permitirá realizar búsquedas por similitud vectorial ultra-rápidas.


Creamos la tabla ARTICLE_EMBEDDINGS. Es importante usar el tipo de dato VECTOR(N), donde N es la dimensionalidad del modelo de embedding (cohere.embed-english-v3.0 suele usar 1024 o 768; asumimos 1024 como estándar aquí).

```
CREATE TABLE ARTICLE_EMBEDDINGS (
    "article_id" NUMBER NOT NULL,
    "vector" VECTOR(1024)
);

```

La función load_embeddings_to_adb inserta masivamente los embeddings calculados en la tabla ARTICLE_EMBEDDINGS. Para ello, eliminamos nulos, convertimos cada vector a array.float32 (formato compatible con columnas VECTOR en ADB) y usamos executemany por lotes para cargar eficientemente los vectores sin saturar el servicio.

In [ ]:
def load_embeddings_to_adb(df_embeddings, batch_size=1000):
    df_clean = df_embeddings.dropna(subset=['embedding_vector'])

    # Convierte cada vector a array.float32
    data_to_insert = [
        (int(row.article_id), array.array("f", row.embedding_vector))
        for row in df_clean.itertuples(index=False)
    ]

    print(f"Iniciando carga masiva de {len(data_to_insert)} vectores a ADB...")

    conn = oracledb.connect(
        user="ADMIN",
        password="TecMonterreyTeam3",
        dsn="team3vectordatabase_high",
        config_dir="/content/wallet"
    )

    insert_sql = 'INSERT INTO ARTICLE_EMBEDDINGS ("article_id", "vector") VALUES (:1, :2)'

    try:
        with conn.cursor() as cursor:
            # meter en chunks para no saturar
            for i in range(0, len(data_to_insert), batch_size):
                chunk = data_to_insert[i:i+batch_size]
                cursor.executemany(insert_sql, chunk)
                conn.commit()
                print(f"   ✅ Insertados {i+len(chunk)}/{len(data_to_insert)}")

        print("✅ Carga masiva de embeddings completada.")
    except Exception as e:
        print(f"CRÍTICO: Fallo en la carga: {e}")
        conn.rollback()
    finally:
        conn.close()

load_embeddings_to_adb(df_embeddings)


Iniciando carga masiva de 105542 vectores a ADB...
   ✅ Insertados 1000/105542
   ✅ Insertados 2000/105542
   ✅ Insertados 3000/105542
   ✅ Insertados 4000/105542
   ✅ Insertados 5000/105542
   ✅ Insertados 6000/105542
   ✅ Insertados 7000/105542
   ✅ Insertados 8000/105542
   ✅ Insertados 9000/105542
   ✅ Insertados 10000/105542
   ✅ Insertados 11000/105542
   ✅ Insertados 12000/105542
   ✅ Insertados 13000/105542
   ✅ Insertados 14000/105542
   ✅ Insertados 15000/105542
   ✅ Insertados 16000/105542
   ✅ Insertados 17000/105542
   ✅ Insertados 18000/105542
   ✅ Insertados 19000/105542
   ✅ Insertados 20000/105542
   ✅ Insertados 21000/105542
   ✅ Insertados 22000/105542
   ✅ Insertados 23000/105542
   ✅ Insertados 24000/105542
   ✅ Insertados 25000/105542
   ✅ Insertados 26000/105542
   ✅ Insertados 27000/105542
   ✅ Insertados 28000/105542
   ✅ Insertados 29000/105542
   ✅ Insertados 30000/105542
   ✅ Insertados 31000/105542
   ✅ Insertados 32000/105542
   ✅ Insertados 33000/105542
 

## 3.4. Entrenamiento del Modelo de Recomendación (Basado en Embeddings)

Construimos el motor de personalización utilizando los embeddings semánticos de cada artículo, generados previamente con OCI Generative AI. A partir de la tabla INTERACTIONS_FOR_CF, identificamos los artículos más representativos del historial de cada cliente y calculamos su perfil vectorial combinando los embeddings correspondientes.

Luego, el modelo recomienda productos midiendo la similitud coseno entre el perfil del usuario y todos los artículos del catálogo, seleccionando los más similares como recomendaciones personalizadas.

**Objetivo**: Generar un archivo de modelo (.pkl) que la aplicación o el chatbot puedan cargar para ofrecer recomendaciones en tiempo real ("Porque compraste X, te recomendamos Y").

### 3.4.A. Obtención del Top de Artículos por Usuario (Fuente para el Perfil Vectorial)


La función `get_top_articles_for_user` consulta la tabla INTERACTIONS_FOR_CF para recuperar los artículos más comprados por un cliente específico. A partir del customer_id, se agrupan sus interacciones históricas y se seleccionan los artículos con mayor purchase_count, con un límite configurable (por defecto, los top 20).

El resultado se devuelve como un DataFrame de Pandas con las columnas `article_id` y `purchase_count`.

In [ ]:
def get_top_articles_for_user(user_id, top_n=20):
    sql = """
        SELECT "article_id", "purchase_count"
        FROM INTERACTIONS_FOR_CF
        WHERE "customer_id" = :user_id
        ORDER BY "purchase_count" DESC, "article_id" ASC
        FETCH FIRST :top_n ROWS ONLY
    """
    rows = query(sql, params={"user_id": user_id, "top_n": top_n})
    df = pd.DataFrame(rows, columns=["article_id", "purchase_count"])
    return df


In [ ]:
get_top_articles_for_user("c5c2d0a9eb5f8b76f087ddb2b8963ed7aac139a833f8bfc35a063b201a3a5185")

,article_id,purchase_count
0,557599026,3
1,598859001,2
2,742982003,2
3,744393002,2
4,156231001,1
5,228257001,1
6,372860024,1
7,373506001,1
8,399223001,1
9,399223025,1


### 3.4.B. Obtención de Embeddings desde la Base de Datos Vectorial

La función `get_article_embeddings` consulta la tabla ARTICLE_EMBEDDINGS para recuperar los vectores semánticos asociados a la lista de artículos. A partir del conjunto de article_ids, la función extrae la columna vector (almacenada en formato VECTOR en ADB) y la convierte directamente en arreglos NumPy. Finalmente, regresa los IDs encontrados y una matriz con los embeddings listos para su uso en cálculos de similitud.


In [ ]:
def get_article_embeddings(article_ids):
    if not article_ids:
        return np.array([]), np.array([])

    # Convert article_ids to a string suitable for SQL IN clause
    # e.g., [123, 456] -> "123,456"
    # e.g., [123] -> "123"
    ids_str = ",".join(str(int(a)) for a in article_ids)

    sql = f"""
        SELECT "article_id", "vector"
        FROM ARTICLE_EMBEDDINGS
        WHERE "article_id" IN ({ids_str})
    """

    rows = query(sql)

    arts = []
    vecs = []

    for article_id, vec in rows:
        # vec ya es un vector: oracle VECTOR
        # Lo convertimos directo a numpy
        v = np.array(vec, dtype=np.float32)

        arts.append(article_id)
        vecs.append(v)

    if not vecs:
        return np.array([]), np.array([])

    return np.array(arts), np.vstack(vecs)

### 3.4.C. Construcción del Perfil Vectorial del Usuario

Con las funciones `build_user_profile_vector` y `get_user_profile`, construimos el vector de perfil de cada usuario a partir de sus artículos más representativos. Primero, recuperamos sus productos más comprados junto con sus embeddings correspondientes. Después, combinamos estos vectores mediante un promedio ponderado, utilizando la frecuencia de compra como peso. Finalmente, normalizamos el vector resultante para obtener un perfil listo para comparaciones mediante similitud coseno.

In [ ]:
def build_user_profile_vector(top_df):
    # matriz (n_articulos, dim)
    item_vecs = np.vstack(top_df["vector"].values)
    weights = top_df["purchase_count"].values.astype(np.float32)

    # promedio ponderado
    user_vec = np.average(item_vecs, axis=0, weights=weights)

    # normalizar (para cosine similarity)
    norm = np.linalg.norm(user_vec)
    if norm > 0:
        user_vec = user_vec / norm

    return user_vec


In [ ]:
def get_user_profile(user_id, top_n=20):
    top_df = get_top_articles_for_user(user_id, top_n=top_n)
    if top_df.empty:
        return None, None  # usuario sin compras

    article_ids = top_df["article_id"].tolist()
    arts, vecs = get_article_embeddings(article_ids)

    # map article_id - vector
    vec_dict = {aid: vec for aid, vec in zip(arts, vecs)}
    top_df["vector"] = top_df["article_id"].map(vec_dict)

    # quitar filas sin embedding
    top_df = top_df.dropna(subset=["vector"])
    if top_df.empty:
        return None, None

    user_vec = build_user_profile_vector(top_df)
    return user_vec, set(article_ids)  # también regresamos los ya comprados


### 3.4.D. Carga de todos los Embeddings de Artículos

Con la función `load_all_article_embeddings`, cargamos todos los vectores semánticos almacenados en la tabla ARTICLE_EMBEDDINGS. Convertimos cada embedding al formato NumPy y construimos una matriz donde cada fila representa un artículo. Posteriormente, normalizamos cada vector para asegurar comparaciones consistentes mediante similitud coseno.

In [ ]:
def load_all_article_embeddings():
    sql = 'SELECT "article_id", "vector" FROM ARTICLE_EMBEDDINGS'
    rows = query(sql)
    arts = []
    vecs = []

    for article_id, vec in rows:
        v = np.array(vec, dtype=np.float32)
        arts.append(article_id)
        vecs.append(v)

    article_ids = np.array(arts)
    matrix = np.vstack(vecs)

    # normalizar cada vector para cosine similarity
    norms = np.linalg.norm(matrix, axis=1, keepdims=True)
    norms[norms == 0] = 1.0
    matrix = matrix / norms

    return article_ids, matrix


### 3.4.E. Cálculo de la Popularidad Global de los Artículos

Con `get_total_purchases`, calculamos la popularidad total de un conjunto de artículos a partir de su historial completo de compras. Para ello, consultamos la tabla INTERACTIONS_FOR_CF y sumamos la frecuencia de compra (purchase_count) de cada artículo. Finalmente, devolvemos un diccionario que mapea cada article_id con su total acumulado. Esto nos es útil para incorporar la métrica de popularidad en las recomendaciones y enriquecer los resultados del modelo.

In [ ]:
def get_total_purchases(article_ids):
    if not article_ids:
        return {}

    id_list = ",".join(str(int(a)) for a in article_ids)

    sql = f"""
        SELECT "article_id", SUM("purchase_count") AS total_purchases
        FROM INTERACTIONS_FOR_CF
        WHERE "article_id" IN ({id_list})
        GROUP BY "article_id"
    """

    rows = query(sql)

    # convertir a diccionario: {article_id: total_purchases}
    totals = {row[0]: row[1] for row in rows}
    return totals


### 3.4.F. Generación de Recomendaciones Personalizadas

Para producir recomendaciones finales, primero obtenemos el vector de preferencias del usuario (`user_vec`) y verificamos que exista suficiente historial para trabajar. Después, cargamos todos los embeddings de artículos y calculamos la similitud coseno entre el perfil del usuario y cada producto usando el producto punto, ya que los vectores están normalizados. Este proceso nos permite identificar rápidamente cuáles artículos son más afines al usuario.
Al ordenar esos puntajes y seleccionar los k valores más altos, generamos una lista de recomendaciones personalizadas acompañadas de su nivel de similitud.


In [ ]:
def recommend_for_user(user_id, k=5, top_n_profile=20):
    # Construimos el vector de gustos del usuario
    user_vec, _ = get_user_profile(user_id, top_n=top_n_profile)
    if user_vec is None:
        print("Usuario sin suficientes datos, usar fallback (populares, etc).")
        return []

    # Cargamos TODOS los embeddings de artículos
    all_ids, all_vecs = load_all_article_embeddings()

    # Cosine similarity = producto punto porque todos los vectores están normalizados (Fuerza bruta)
    sims = all_vecs @ user_vec      # shape (N,)

    # Sacamos el top K global
    top_idx = np.argsort(-sims)[:k]
    rec_ids = all_ids[top_idx]
    rec_scores = sims[top_idx]

    return list(zip(rec_ids, rec_scores))


### 3.4.G. Ejemplo de funcionamiento del modelo


In [ ]:
def run_example():
    user_id = "0e5c9909c03c8a37991870114712c46b3ae0e7902220d8c4bb2a5390f35fed43"

    print("\n=== TOP 20 COMPRAS DEL USUARIO ===")
    top_df = get_top_articles_for_user(user_id, top_n=20)
    print(top_df)

    print("\n=== TOP 5 RECOMENDADOS (POR SIMILITUD DE EMBEDDINGS) ===")
    recs = recommend_for_user(user_id, k=5, top_n_profile=20)

    # sacar solo los article_ids recomendados
    rec_article_ids = [art_id for art_id, score in recs]

    # obtener purchase_count global
    totals = get_total_purchases(rec_article_ids)

    for rank, (art_id, score) in enumerate(recs, start=1):
        total_bought = totals.get(art_id, 0)
        print(f"{rank}. ARTICLE_ID = {art_id} | similarity = {score:.4f} | total_purchased = {total_bought}")


In [ ]:
run_example()


=== TOP 20 COMPRAS DEL USUARIO ===
   article_id  purchase_count
0   436083002               1
1   529008011               1
2   547365012               1
3   568594001               1
4   666080003               1
5   671800003               1
6   681176012               1
7   695071003               1
8   720687008               1

=== TOP 5 RECOMENDADOS (POR SIMILITUD DE EMBEDDINGS) ===
1. ARTICLE_ID = 639965002 | similarity = 0.8515 | total_purchased = 387
2. ARTICLE_ID = 567874005 | similarity = 0.8507 | total_purchased = 127
3. ARTICLE_ID = 567874006 | similarity = 0.8503 | total_purchased = 245
4. ARTICLE_ID = 567874001 | similarity = 0.8494 | total_purchased = 8
5. ARTICLE_ID = 567874004 | similarity = 0.8488 | total_purchased = 5


### 3.4.H. Exportación del Modelo a OCI Object Storage

Para que el chatbot pueda utilizar el motor de recomendación sin reconstruirlo desde cero, almacenamos los componentes clave del sistema (vectores de usuario, embeddings de artículos y funciones auxiliares) en archivos locales. Estos archivos se suben posteriormente a un bucket de OCI Object Storage, permitiendo que la aplicación cargue el modelo y el conjunto completo de embeddings directamente desde la nube, reduciendo tiempos de inicialización y evitando reprocesamientos innecesarios.

In [ ]:
# Cargamos todos los embeddings de artículos
article_ids, article_vectors = load_all_article_embeddings()

# Construimos dict del modelo
model_data = {
    "article_ids": article_ids,
    "article_vectors": article_vectors,
}

# Guardamos localmente
import joblib

MODEL_FILENAME = 'recommendation_model_v1.pkl'
LOCAL_MODEL_PATH = f'/content/{MODEL_FILENAME}'

print(f"Guardando modelo en {LOCAL_MODEL_PATH}...")
joblib.dump(model_data, LOCAL_MODEL_PATH)
print("Modelo guardado localmente")


Guardando modelo en /content/recommendation_model_v1.pkl...
Modelo guardado localmente


In [ ]:
import io

print(f"Subiendo modelo a OCI/{MODEL_FILENAME}...")

try:
    # Leer el archivo local
    with open(LOCAL_MODEL_PATH, 'rb') as f:
        object_storage_client.put_object(
            NAMESPACE,
            BUCKET_NAME,
            f"models/{MODEL_FILENAME}",
            f
        )

    print("Modelo actualizado en la nube")

except Exception as e:
    print("Error al subir modelo:", e)


Subiendo modelo a OCI/recommendation_model_v1.pkl...
Modelo actualizado en la nube


# Desde aqui

In [ ]:
import numpy as np
import pandas as pd

def get_sample_users(num_users=100, min_purchases=3):
    """
    Devuelve una lista de customer_id con al menos `min_purchases`
    interacciones, en orden aleatorio, limitado a num_users.
    """
    sql = f"""
        SELECT DISTINCT "customer_id"
        FROM INTERACTIONS_FOR_CF
        GROUP BY "customer_id"
        HAVING SUM("purchase_count") >= :min_purchases
        ORDER BY dbms_random.value
        FETCH FIRST :num_users ROWS ONLY
    """
    rows = query(sql, params={"min_purchases": min_purchases, "num_users": num_users})
    user_ids = [r[0] for r in rows]
    return user_ids


In [ ]:
def evaluate_avg_topk_similarity(num_users=100, k=5, top_n_profile=20, min_purchases=3):
    """
    1) Toma una muestra de usuarios.
    2) Para cada uno, genera recomendaciones top-k.
    3) Calcula el promedio de similarity de esos top-k.
    4) Devuelve estadísticas agregadas.
    """
    user_ids = get_sample_users(num_users=num_users, min_purchases=min_purchases)
    if not user_ids:
        print("No se encontraron usuarios con suficiente historial.")
        return None

    all_user_means = []
    all_scores = []

    users_with_recs = 0

    for uid in user_ids:
        recs = recommend_for_user(uid, k=k, top_n_profile=top_n_profile)

        # recs: lista de (article_id, score)
        if not recs:
            # usuario sin perfil o sin recomendaciones
            continue

        users_with_recs += 1
        scores = [score for _, score in recs]

        user_mean = float(np.mean(scores))
        all_user_means.append(user_mean)
        all_scores.extend(scores)

        print(f"Usuario: {uid}")
        for rank, (art_id, score) in enumerate(recs, start=1):
            print(f"  {rank}. ARTICLE_ID = {art_id} | similarity = {score:.4f}")
        print(f"  -> mean_top{k}_sim = {user_mean:.4f}")
        print("-" * 60)

    if users_with_recs == 0:
        print("Ningún usuario evaluado generó recomendaciones (probablemente sin historial suficiente).")
        return None

    overall_mean = float(np.mean(all_scores))
    mean_of_means = float(np.mean(all_user_means))
    std_of_means = float(np.std(all_user_means))

    print("\n===== RESUMEN GLOBAL =====")
    print(f"Usuarios solicitados: {num_users}")
    print(f"Usuarios con recomendaciones: {users_with_recs}")
    print(f"Promedio global de similarity de todos los top-{k}: {overall_mean:.4f}")
    print(f"Promedio de 'mean_top{k}_sim' por usuario: {mean_of_means:.4f}")
    print(f"Desviación estándar entre usuarios (mean_top{k}_sim): {std_of_means:.4f}")

    results = {
        "num_users_requested": num_users,
        "num_users_with_recs": users_with_recs,
        "overall_mean_topk_similarity": overall_mean,
        "mean_of_user_means": mean_of_means,
        "std_of_user_means": std_of_means,
        "user_means": all_user_means,
    }
    return results


In [ ]:
results = evaluate_avg_topk_similarity(
    num_users=100,
    k=5,
    top_n_profile=20,
    min_purchases=3
)


Usuario: 43a3604f70493be51a9bd974937f0e8b5fe76eeabd7f157bfc9dfee7fdb09565
  1. ARTICLE_ID = 676728001 | similarity = 0.8646
  2. ARTICLE_ID = 622159001 | similarity = 0.8625
  3. ARTICLE_ID = 874390004 | similarity = 0.8623
  4. ARTICLE_ID = 856109002 | similarity = 0.8611
  5. ARTICLE_ID = 667845003 | similarity = 0.8605
  -> mean_top5_sim = 0.8622
------------------------------------------------------------
Usuario: fe634082cde60ced5c9fc04017ec29a7332a5669ba0a8a169be0667c9036620c
  1. ARTICLE_ID = 757582003 | similarity = 0.8973
  2. ARTICLE_ID = 816431001 | similarity = 0.8961
  3. ARTICLE_ID = 696896002 | similarity = 0.8929
  4. ARTICLE_ID = 791587005 | similarity = 0.8914
  5. ARTICLE_ID = 696896001 | similarity = 0.8904
  -> mean_top5_sim = 0.8936
------------------------------------------------------------
Usuario: 4713a2b4950f284ece0659dd44b43dadb8e9c6a563fe306af0f4865075411055
  1. ARTICLE_ID = 868064001 | similarity = 0.9654
  2. ARTICLE_ID = 868064008 | similarity = 0.9650


HitRate@5 con “holdout” de un artículo comprado

In [ ]:
def get_top_popular_articles(k=50):
    """
    Devuelve los top-k artículos más comprados globalmente.
    Usaremos esto como baseline de popularidad.
    """
    sql = f"""
        SELECT "article_id", SUM("purchase_count") AS total_purchases
        FROM INTERACTIONS_FOR_CF
        GROUP BY "article_id"
        ORDER BY total_purchases DESC, "article_id" ASC
        FETCH FIRST :k ROWS ONLY
    """
    rows = query(sql, params={"k": k})
    popular_article_ids = [r[0] for r in rows]
    return popular_article_ids

In [ ]:
def evaluate_popularity_baseline_recall_ndcg_at_k(
    num_users=100,
    k=5,
    top_n_profile=50,
    max_holdout=3,
    min_purchases=3
):
    """
    Baseline de POPULARIDAD:
      - Para todos los usuarios recomienda SIEMPRE el mismo top-K
        de artículos más comprados globalmente.
      - Evalúa Recall@K y NDCG@K usando multi-holdout por usuario.
    """

    # Top-K artículos globales
    popular_ids = get_top_popular_articles(k=k)
    if not popular_ids:
        print("No se pudo obtener el top de artículos populares.")
        return None

    # Usuarios a evaluar
    user_ids = get_sample_users(num_users=num_users, min_purchases=min_purchases)
    if not user_ids:
        print("No se encontraron usuarios con suficiente historial.")
        return None

    user_recalls = []
    user_ndcgs = []
    total_users_eval = 0

    for uid in user_ids:
        # Holdouts para este usuario
        holdout_items = get_multiple_holdout_items_for_user(
            uid,
            top_n=top_n_profile,
            max_holdout=max_holdout
        )

        if not holdout_items:
            continue

        holdout_set = set(holdout_items)

        # Recos del baseline: mismos para todos
        rec_ids = popular_ids[:k]

        # Relevancias binaria: 1 si rec_id está en holdout
        relevances = [1 if rid in holdout_set else 0 for rid in rec_ids]

        hits = sum(relevances)
        recall_k = hits / len(holdout_items)
        ndcg_k = ndcg_at_k(relevances, k)

        user_recalls.append(recall_k)
        user_ndcgs.append(ndcg_k)
        total_users_eval += 1

        print(f"[POPULARIDAD] Usuario: {uid}")
        print(f"  Holdout items: {holdout_items}")
        print(f"  Recs top-{k} populares: {list(rec_ids)}")
        print(f"  Relevances: {relevances}")
        print(f"  Recall@{k}: {recall_k:.4f} | NDCG@{k}: {ndcg_k:.4f}")
        print("-" * 60)

    if total_users_eval == 0:
        print("No fue posible evaluar ningún usuario para el baseline de popularidad.")
        return None

    mean_recall = float(np.mean(user_recalls))
    mean_ndcg = float(np.mean(user_ndcgs))

    print("\n===== RESUMEN BASELINE POPULARIDAD =====")
    print(f"Usuarios evaluados: {total_users_eval}")
    print(f"Recall@{k} medio: {mean_recall:.4f}")
    print(f"NDCG@{k} medio: {mean_ndcg:.4f}")

    results = {
        "num_users_evaluated": total_users_eval,
        "mean_recall_at_k": mean_recall,
        "mean_ndcg_at_k": mean_ndcg,
        "k": k,
        "user_recalls": user_recalls,
        "user_ndcgs": user_ndcgs,
    }
    return results

In [ ]:
pop_baseline = evaluate_popularity_baseline_recall_ndcg_at_k(
    num_users=100,
    k=5,
    top_n_profile=50,
    max_holdout=3,
    min_purchases=3
)

[POPULARIDAD] Usuario: 920b7e304072d7fba974b10623a235632ef29055f88b6be222751963d570198b
  Holdout items: [880238003, 903924002]
  Recs top-5 populares: [706016001, 706016002, 372860001, 610776002, 759871002]
  Relevances: [0, 0, 0, 0, 0]
  Recall@5: 0.0000 | NDCG@5: 0.0000
------------------------------------------------------------
[POPULARIDAD] Usuario: 21a1682acff1d09338d2be9db94dd979401a8cca3b73145ea4683ee5a182e5a2
  Holdout items: [562245001, 706016053, 678942001]
  Recs top-5 populares: [706016001, 706016002, 372860001, 610776002, 759871002]
  Relevances: [0, 0, 0, 0, 0]
  Recall@5: 0.0000 | NDCG@5: 0.0000
------------------------------------------------------------
[POPULARIDAD] Usuario: ddee9fdff4df0d8740869a58b486f6b22fcc4822b941c37a7d067ae180898090
  Holdout items: [660290001, 577375001, 567805005]
  Recs top-5 populares: [706016001, 706016002, 372860001, 610776002, 759871002]
  Relevances: [0, 0, 0, 0, 0]
  Recall@5: 0.0000 | NDCG@5: 0.0000
----------------------------------

In [ ]:
def get_all_article_ids():
    """
    Devuelve todos los article_id presentes en ARTICLE_EMBEDDINGS.
    Aprovecha que ya tienes la tabla vectorial.
    """
    sql = 'SELECT "article_id" FROM ARTICLE_EMBEDDINGS'
    rows = query(sql)
    all_ids = [r[0] for r in rows]
    return all_ids

In [ ]:
import random

def evaluate_random_baseline_recall_ndcg_at_k(
    num_users=100,
    k=5,
    top_n_profile=50,
    max_holdout=3,
    min_purchases=3
):
    """
    Baseline ALEATORIO:
      - Para cada usuario elige K artículos distintos al azar del catálogo.
      - Evalúa Recall@K y NDCG@K con multi-holdout.
    """

    all_article_ids = get_all_article_ids()
    if len(all_article_ids) < k:
        print("No hay suficientes artículos en el catálogo para hacer random@K.")
        return None

    all_article_ids = list(all_article_ids)

    user_ids = get_sample_users(num_users=num_users, min_purchases=min_purchases)
    if not user_ids:
        print("No se encontraron usuarios con suficiente historial.")
        return None

    user_recalls = []
    user_ndcgs = []
    total_users_eval = 0

    for uid in user_ids:
        holdout_items = get_multiple_holdout_items_for_user(
            uid,
            top_n=top_n_profile,
            max_holdout=max_holdout
        )
        if not holdout_items:
            continue

        holdout_set = set(holdout_items)

        # Muestreamos K artículos aleatorios del catálogo
        rec_ids = random.sample(all_article_ids, k)

        relevances = [1 if rid in holdout_set else 0 for rid in rec_ids]

        hits = sum(relevances)
        recall_k = hits / len(holdout_items)
        ndcg_k = ndcg_at_k(relevances, k)

        user_recalls.append(recall_k)
        user_ndcgs.append(ndcg_k)
        total_users_eval += 1

        print(f"[RANDOM] Usuario: {uid}")
        print(f"  Holdout items: {holdout_items}")
        print(f"  Recs top-{k} random: {list(rec_ids)}")
        print(f"  Relevances: {relevances}")
        print(f"  Recall@{k}: {recall_k:.4f} | NDCG@{k}: {ndcg_k:.4f}")
        print("-" * 60)

    if total_users_eval == 0:
        print("No fue posible evaluar ningún usuario para el baseline random.")
        return None

    mean_recall = float(np.mean(user_recalls))
    mean_ndcg = float(np.mean(user_ndcgs))

    print("\n===== RESUMEN BASELINE RANDOM =====")
    print(f"Usuarios evaluados: {total_users_eval}")
    print(f"Recall@{k} medio: {mean_recall:.4f}")
    print(f"NDCG@{k} medio: {mean_ndcg:.4f}")

    results = {
        "num_users_evaluated": total_users_eval,
        "mean_recall_at_k": mean_recall,
        "mean_ndcg_at_k": mean_ndcg,
        "k": k,
        "user_recalls": user_recalls,
        "user_ndcgs": user_ndcgs,
    }
    return results

In [ ]:
rand_baseline = evaluate_random_baseline_recall_ndcg_at_k(
    num_users=100,
    k=5,
    top_n_profile=50,
    max_holdout=3,
    min_purchases=3
)

[RANDOM] Usuario: 605291ffcb04f8808a37abb56bdfa4d55c83c9c66fd47f89c9bfc0f9b6263bfb
  Holdout items: [446224014, 446224016, 700477001]
  Recs top-5 random: [569248014, 735006001, 731563001, 810921004, 798883007]
  Relevances: [0, 0, 0, 0, 0]
  Recall@5: 0.0000 | NDCG@5: 0.0000
------------------------------------------------------------
[RANDOM] Usuario: 237afc9d0cb933793a2c712fa49a8ccfcada669d5d5596d5603b31cb10dd1102
  Holdout items: [898439003, 891663002, 784650008]
  Recs top-5 random: [770816004, 794053002, 615130001, 864043003, 694124002]
  Relevances: [0, 0, 0, 0, 0]
  Recall@5: 0.0000 | NDCG@5: 0.0000
------------------------------------------------------------
[RANDOM] Usuario: fa1bc023f0ef3813376d75df543353d94a7994cb2aa10b74d33c11579f9b41ea
  Holdout items: [579541072, 757671007, 677076001]
  Recs top-5 random: [805510033, 503834001, 870942001, 548613032, 868060005]
  Relevances: [0, 0, 0, 0, 0]
  Recall@5: 0.0000 | NDCG@5: 0.0000
-----------------------------------------------

In [ ]:
def run_full_comparison(
    num_users=100,
    k=5,
    top_n_profile=50,
    max_holdout=3,
    min_purchases=3
):
    print(f"\n\n=========== EVALUANDO MODELO DE EMBEDDINGS (k={k}) ===========")
    model_metrics = evaluate_recall_ndcg_at_k(
        num_users=num_users,
        k=k,
        top_n_profile=top_n_profile,
        max_holdout=max_holdout,
        min_purchases=min_purchases
    )

    print(f"\n\n=========== EVALUANDO BASELINE POPULARIDAD (k={k}) ===========")
    pop_metrics = evaluate_popularity_baseline_recall_ndcg_at_k(
        num_users=num_users,
        k=k,
        top_n_profile=top_n_profile,
        max_holdout=max_holdout,
        min_purchases=min_purchases
    )

    print(f"\n\n=========== EVALUANDO BASELINE RANDOM (k={k}) ===========")
    rand_metrics = evaluate_random_baseline_recall_ndcg_at_k(
        num_users=num_users,
        k=k,
        top_n_profile=top_n_profile,
        max_holdout=max_holdout,
        min_purchases=min_purchases
    )

    print("\n\n===================== RESUMEN COMPARATIVO =====================")
    def safe_get(d, key, default=None):
        return d.get(key, default) if d is not None else default

    print(f"k = {k}")
    print("Modelo embeddings:")
    print(f"  Recall@{k}: {safe_get(model_metrics, 'mean_recall_at_k', 0):.4f}")
    print(f"  NDCG@{k}:  {safe_get(model_metrics, 'mean_ndcg_at_k', 0):.4f}")

    print("Popularidad:")
    print(f"  Recall@{k}: {safe_get(pop_metrics, 'mean_recall_at_k', 0):.4f}")
    print(f"  NDCG@{k}:  {safe_get(pop_metrics, 'mean_ndcg_at_k', 0):.4f}")

    print("Random:")
    print(f"  Recall@{k}: {safe_get(rand_metrics, 'mean_recall_at_k', 0):.4f}")
    print(f"  NDCG@{k}:  {safe_get(rand_metrics, 'mean_ndcg_at_k', 0):.4f}")

    return {
        "model": model_metrics,
        "popularity": pop_metrics,
        "random": rand_metrics,
    }

In [ ]:
results_k5 = run_full_comparison(
    num_users=100,
    k=5,
    top_n_profile=50,
    max_holdout=3,
    min_purchases=3
)

results_k20 = run_full_comparison(
    num_users=100,
    k=20,
    top_n_profile=50,
    max_holdout=3,
    min_purchases=3
)



=========== EVALUANDO MODELO DE EMBEDDINGS (k=5) ===========
Usuario: 7572672bc8a17c71cbc09fb98ab4504afc81efd2e8a76a2284de8254d378dc33
  Holdout items: [629381002, 688290001]
  Recs top-5: [629381003, 629381001, 629381012, 629381004, 629381002]
  Relevances: [0, 0, 0, 0, 1]
  Recall@5: 0.5000 | NDCG@5: 0.3869
------------------------------------------------------------
Usuario: 4039166526099b7afa2c5a962b0ce732ecf20dcd1581735b70718a1508bba90c
  Holdout items: [554598001, 803949001, 797710001]
  Recs top-5: [602722001, 602722002, 699580004, 871521003, 617900007]
  Relevances: [0, 0, 0, 0, 0]
  Recall@5: 0.0000 | NDCG@5: 0.0000
------------------------------------------------------------
Usuario: 6d1176c4f950c26f6435bd7624bbfd8bfeaa3d3fd844dff3473682c9ba0334ac
  Holdout items: [674071001, 695263001, 355569001]
  Recs top-5: [908489004, 908489006, 556415001, 783645001, 669811001]
  Relevances: [0, 0, 0, 0, 0]
  Recall@5: 0.0000 | NDCG@5: 0.0000
-------------------------------------------

In [ ]:
def evaluate_model_coverage_at_k(
    num_users=100,
    k=5,
    top_n_profile=50,
    min_purchases=3
):
    """
    Calcula Coverage@K para el modelo de embeddings:
      coverage = (# artículos distintos recomendados en top-K) / (# artículos en catálogo)
    """
    # Catálogo completo
    all_article_ids, all_article_vecs = load_all_article_embeddings()
    all_article_ids = np.array(all_article_ids)
    total_catalog = len(all_article_ids)

    if total_catalog == 0:
        print("Catálogo vacío, no se puede calcular coverage.")
        return None

    # Usuarios
    user_ids = get_sample_users(num_users=num_users, min_purchases=min_purchases)
    if not user_ids:
        print("No hay usuarios suficientes.")
        return None

    recommended_items = set()
    total_users_eval = 0

    for uid in user_ids:
        # Usamos el perfil normal (sin holdout, aquí solo nos importa la recomendación)
        user_vec, _ = get_user_profile(uid, top_n=top_n_profile)
        if user_vec is None:
            continue

        # Similaridad y top-K
        sims = all_article_vecs @ user_vec
        top_idx = np.argsort(-sims)[:k]
        rec_ids = all_article_ids[top_idx]

        recommended_items.update(rec_ids)
        total_users_eval += 1

    if total_users_eval == 0:
        print("No fue posible recomendar para ningún usuario.")
        return None

    coverage = len(recommended_items) / total_catalog

    print("\n===== COBERTURA MODELO EMBEDDINGS =====")
    print(f"Usuarios evaluados: {total_users_eval}")
    print(f"Artículos distintos recomendados (top-{k}): {len(recommended_items)}")
    print(f"Catálogo total: {total_catalog}")
    print(f"Coverage@{k}: {coverage:.4f}")

    return {
        "num_users_evaluated": total_users_eval,
        "distinct_items_recommended": len(recommended_items),
        "total_catalog": total_catalog,
        "coverage_at_k": coverage,
        "k": k,
    }

In [ ]:
coverage_model_k5 = evaluate_model_coverage_at_k(num_users=100, k=5)
coverage_model_k20 = evaluate_model_coverage_at_k(num_users=100, k=20)


===== COBERTURA MODELO EMBEDDINGS =====
Usuarios evaluados: 100
Artículos distintos recomendados (top-5): 364
Catálogo total: 105542
Coverage@5: 0.0034

===== COBERTURA MODELO EMBEDDINGS =====
Usuarios evaluados: 100
Artículos distintos recomendados (top-20): 1228
Catálogo total: 105542
Coverage@20: 0.0116


In [ ]:
def evaluate_model_popularity_bias(
    num_users=100,
    k=5,
    top_n_profile=50,
    min_purchases=3
):
    """
    Mide el sesgo a popularidad del modelo:
      - Popularidad media de los artículos recomendados
      - Comparada con la popularidad media del catálogo
    """
    # Catálogo completo
    all_article_ids, all_vecs = load_all_article_embeddings()
    all_article_ids = np.array(all_article_ids)
    total_catalog = len(all_article_ids)

    if total_catalog == 0:
        print("Catálogo vacío.")
        return None

    # Popularidad global del catálogo
    catalog_pop_dict = get_total_purchases(all_article_ids)
    catalog_pop_values = [catalog_pop_dict.get(aid, 0) for aid in all_article_ids]
    mean_pop_catalog = float(np.mean(catalog_pop_values))

    # Usuarios
    user_ids = get_sample_users(num_users=num_users, min_purchases=min_purchases)
    if not user_ids:
        print("No hay usuarios suficientes.")
        return None

    rec_pop_values = []
    total_users_eval = 0

    for uid in user_ids:
        # perfil normal del usuario
        user_vec, _ = get_user_profile(uid, top_n=top_n_profile)
        if user_vec is None:
            continue

        # score = similarity contra todos los artículos
        sims = all_vecs @ user_vec
        top_idx = np.argsort(-sims)[:k]
        rec_ids = all_article_ids[top_idx]

        # popularidad de los recomendados
        for rid in rec_ids:
            rec_pop_values.append(catalog_pop_dict.get(rid, 0))

        total_users_eval += 1

    if total_users_eval == 0 or not rec_pop_values:
        print("No se generaron recomendaciones suficientes.")
        return None

    mean_pop_recs = float(np.mean(rec_pop_values))

    print("\n===== SESGO A POPULARIDAD MODELO EMBEDDINGS =====")
    print(f"Usuarios evaluados: {total_users_eval}")
    print(f"Popularidad media del catálogo:     {mean_pop_catalog:.2f}")
    print(f"Popularidad media recomendaciones:  {mean_pop_recs:.2f}")

    return {
        "num_users_evaluated": total_users_eval,
        "mean_popularity_catalog": mean_pop_catalog,
        "mean_popularity_recommendations": mean_pop_recs,
        "k": k,
    }


# ===============================================================
# 🔥 LLAMADA AUTOMÁTICA AL MÉTODO (esta parte lo ejecuta de verdad)
# ===============================================================

pop_bias_results = evaluate_model_popularity_bias(
    num_users=100,
    k=5,
    top_n_profile=50,
    min_purchases=3
)

print("\n===== RESULTADO devuelto =====")
print(pop_bias_results)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()